<a href="https://colab.research.google.com/github/WilliamStanford/CS790-TopoAE_Classification/blob/main/audio_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import shutil
import glob
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-zk55nnku/demucs_67fdc819aa284ba3a14ca13c7ccdd25e
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-zk55nnku/demucs_67fdc819aa284ba3a14ca13c7ccdd25e
  Resolved https://github.com/facebookresearch/demucs to commit 0fa7d4511b0fca3f96f59fa37f7dcaed555990d6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.8 MB/s e

In [3]:
!pip install pytube pydub librosa torchaudio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install -U openai-whisper

In [4]:
!pip install torch==2.0.0+cu118 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 845.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89989 sha256=c3eb4f5aba6c2b29825254fff02f1460e7de3d823e8f32b038ca7e7e6a021789
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b44fe0dea1bb35234b09f2c22fb8ebb308679c922
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled to

In [36]:
#@title audio utils - store in separate audio_utils.py file
import librosa
import os
from scipy.signal import find_peaks
from pytube import YouTube
from pydub import AudioSegment
import shutil
import io
from pathlib import Path
import subprocess as sp
from typing import Dict, Tuple, Optional, IO
from scipy.io import wavfile
import sys
import select
import numpy as np
import pandas as pd
import secrets
import random



def download_yt_audio(yt_link, project_path, filename):

  yt = YouTube(yt_link)
  video = yt.streams.filter(only_audio=True).first()
  out_file = video.download(output_path=project_path+filename)

  base, ext = os.path.splitext(out_file)

  new_file = '/'.join(out_file.split('/')[:-1])+'.mp3'
  os.rename(out_file, new_file)
  shutil.rmtree(project_path+filename+'/')

  return new_file

def cut_audio(input_path, output_path, max_duration=30):
    audio = AudioSegment.from_file(input_path)

    if len(audio) > max_duration:
        audio = audio[:max_duration]

    audio.export(output_path, format="mp3")

    return output_path


def find_files(extensions, in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()


def get_files(directory, full_dir=False):

  file_list = []
  for root, _, filenames in os.walk(directory):
    for filename in filenames:
      if full_dir==True:
        file_list.append(os.path.join(root, filename))
      else:
        file_list.append(os.path.join(root, filename).split('/')[-1])

  return file_list


def convert_audio_for_model(user_file, EXPECTED_SAMPLE_RATE):
  audio = AudioSegment.from_file(user_file)
  audio = audio.set_frame_rate(EXPECTED_SAMPLE_RATE).set_channels(1)
  output_file = user_file.split('.')[0]+'_converted_audio_file.wav'
  audio.export(output_file, format="wav")
  return output_file


def convert_and_plot_spec(file, EXPECTED_SAMPLE_RATE=16382, fps=30):

  converted_audio_file = convert_audio_for_model(file, EXPECTED_SAMPLE_RATE)

  # Loading audio samples from the wav file:
  sample_rate, audio_samples = wavfile.read(converted_audio_file, 'rb')

  MAX_ABS_INT16 = 32768.0

  x_stft = np.abs(librosa.stft(audio_samples / MAX_ABS_INT16, n_fft=2048, hop_length=int(sample_rate/fps)))
  x_stft_db = librosa.amplitude_to_db(x_stft, ref=np.max)
  spec_df = pd.DataFrame(x_stft_db)

  return x_stft_db, spec_df


def extract_amplitude(data, quantile=.725):

  data = data.copy(deep=True)
  data = data + abs(data.min().min())

  q = np.quantile(data.iloc[:,:], quantile)
  data[data < q] = 0

  sig = data.sum(axis=0)
  #sig = sig/sig.max()

  return pd.DataFrame(sig)


def compile_timeseries(base_filename, audio_path, fps):

  source_sep_path = audio_path.split('.')[0]

  audio_files = get_files(audio_path, full_dir=True)
  audio_files.append(audio_path)

  ts_df = pd.DataFrame([])

  for filename in audio_files:

    EXPECTED_SAMPLE_RATE = 16382

    if '.mp3' in filename:

      x_stft_db, spec_df = convert_and_plot_spec(filename, EXPECTED_SAMPLE_RATE, fps=fps)
      spec_df = spec_df + abs(spec_df.min().min())

      ts_df[filename.split('/')[-1]] = extract_amplitude(spec_df)

      if 'drums' in filename:
        ts_df[filename.split('/')[-1]] = ts_df[filename.split('/')[-1]].rolling(2).mean()
      else:
        ts_df[filename.split('/')[-1]] = ts_df[filename.split('/')[-1]].rolling(3).mean()

  ts_df['background_music.mp3'] = ts_df['bass.mp3'] + ts_df['other.mp3']

  ts_df.dropna(inplace=True)
  ts_df.reset_index(inplace=True)

  for col in ts_df.columns:
    ts_df[col] = ts_df[col]/ts_df[col].max()


  ts_df['starting_visuals'] = np.ones(ts_df.shape[0])

  ts_df = ts_df[['starting_visuals', 'background_music.mp3', 'vocals.mp3']]

  return ts_df

def extract_spectrogram(filepath, fps):

  if filepath.split('.')[-1] == 'mp3':
    converted_audio_file = convert_audio_for_model(filepath)
  else:
    converted_audio_file = filepath

  sample_rate, audio_samples = wavfile.read(converted_audio_file, 'rb')
  MAX_ABS_INT16 = 32768.0

  x_stft = np.abs(librosa.stft(audio_samples / MAX_ABS_INT16, n_fft=2048, hop_length=int(sample_rate/fps)))
  x_stft_db = librosa.amplitude_to_db(x_stft, ref=np.max)
  spec_df = pd.DataFrame(x_stft_db)

  spec_df = spec_df + abs(spec_df.min().min())

  return spec_df


def separate(extensions, inp=None, outp=None):

    inp = inp
    outp = outp
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(extensions, inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")



def get_bpm(filename):

  y, sr = librosa.load(filename, duration=60)
  onset_env = librosa.onset.onset_strength(y=y, sr=sr)
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

  return tempo

def get_beats(filename, spec_df, freq_start=4, freq_end=12, window=3, threshold=300):

  spec_df = spec_df.copy(deep=True)

  sig = spec_df.iloc[freq_start:freq_end,:].sum(axis=0).to_numpy()
  peaks, _ = find_peaks(sig, distance=window, height=threshold)
  beats_df = pd.DataFrame(peaks)

  return beats_df

def get_zoom_speed_via_bpm(bpm):

  bpm_min = 80
  bpm_max = 160
  bpm_diff = bpm_max - bpm_min

  zoom_min = 1.01
  zoom_max = 1.02
  zoom_diff = zoom_max - zoom_min

  if bpm < bpm_min:
    bpm = bpm_min
  if bpm > bpm_max:
    bpm = bpm_max

  zoom = (((bpm - bpm_min) / bpm_diff) * zoom_diff ) + zoom_min

  return zoom

def clip(value, lower, upper):
    return lower if value < lower else upper if value > upper else value

def get_video_params(ts_df, beats_df, bpm, fps, lyrics_dic=None, use_beats=True, use_lyrics=True):

  ts_df = ts_df.copy(deep=True)
  beats_df = beats_df.copy(deep=True)
  params_df = pd.DataFrame(index=ts_df.index)

  beat_scale = get_zoom_speed_via_bpm(bpm)



  args = {
      'seed':0,
      'guidance_scale':7.5,
      'strength':1,
      'steps':30,
      'base_steps':30,
      'base_zoom_rate':.99,
      'zoom_rate':.99,
      'min_zoom':.98,
      'max_zoom':1.01,
      'rotation_rate':0,
      'translation_x':0,
      'translation_y':0,
      'zoom_pos':'center',
      'early_steps':10,
      'base_rotation_rate':3,
      'max_rotation_rate':4,
      'base_noise_amount':0.06,
      'noise_amount':0.06,
      'base_strength':0.6,
      'beat_strength':0.8,
      'base_translation_rate':7,
      'max_translation_rate':10,
      'decay':.99,
      'beat_scale':1.01,
      'last_beat_counter':0,
  }


  params_df['beats'] = 0
  for ind in beats_df.index:
    beat_ind = beats_df.loc[ind, 0]
    params_df.loc[params_df.index==beat_ind, 'beats'] = 1

  #params_df['sentence_start'] = 0

  if lyrics_dic != None:
    for ind, seg in enumerate(lyrics_dic['segments']):
      start = seg['start']*fps
      params_df.loc[params_df.index==int(start), 'sentence_start'] = 1


  last_sentence_start = params_df['sentence_start'].where(params_df['sentence_start']==1).last_valid_index()
  last_beat = params_df['beats'].where(params_df['beats']==1).last_valid_index()

  if use_beats and use_lyrics:
    last_scene_change = max([last_sentence_start, last_beat])
  elif use_beats and not use_lyrics:
    last_scene_change = last_beat
  else:
    last_scene_change = last_sentence_start


  params_df['mov_av_audio'] = ts_df['background_music.mp3'].rolling(5).mean()
  params_df.fillna(value=0, inplace=True)

  #args['last_beat_counter'] = 0

  for ind in params_df.index:

    args['zoom_rate'] = args['zoom_rate']
    args['noise_amount'] = 0
    if ind == 0:
      args['strength'] = 1
    else:
      args['strength'] = args['base_strength']

    if ind % 2 == 0:
      args['seed'] = secrets.randbelow(1_000_000_000)

    if 'sentence_start' in params_df.columns:
      if ind > 0:
        if params_df.loc[ind, 'sentence_start'] == 1:

          args['seed'] = secrets.randbelow(1_000_000_000)
          args['strength'] = args['beat_strength']
          args['steps'] = args['base_steps']
          args['rotation_rate'] = random.uniform(-args['base_rotation_rate'], args['base_rotation_rate'])
          args['translation_x'] = random.randint(-args['base_translation_rate'], args['base_translation_rate'])
          args['translation_y'] = random.randint(-args['base_translation_rate'], args['base_translation_rate'])
          args['noise_amount'] = args['base_noise_amount']


    if use_beats:
      if params_df.loc[ind, 'beats'] == 1:

        #args['last_beat_counter']  = 1

        args['seed'] = secrets.randbelow(1_000_000_000)
        args['strength'] = args['beat_strength']
        args['steps'] = args['base_steps']
        args['rotation_rate'] = random.uniform(-args['base_rotation_rate'], args['base_rotation_rate'])
        args['translation_x'] = random.randint(-args['base_translation_rate'], args['base_translation_rate'])
        args['translation_y'] = random.randint(-args['base_translation_rate'], args['base_translation_rate'])
        args['noise_amount'] = args['base_noise_amount']
        args['zoom_rate'] = random.uniform(args['min_zoom'], args['max_zoom'])

        #chance = params_df.loc[ind, 'mov_av_audio']
        #draw = np.random.uniform(0,1,2)

        #if draw[0] < chance:

        #    args['rotation_rate'] = random.uniform(-args['max_rotation_rate'], args['max_rotation_rate'])

            #add_rot = random.uniform(-args['base_rotation_rate'], args['base_rotation_rate'])*(chance+.25)
            #args['rotation_rate']  = args['rotation_rate'] + add_rot
            #args['rotation_rate']  = clip(args['rotation_rate'], -args['max_rotation_rate'], args['max_rotation_rate'])

        #if draw[1] < chance:


        #    args['translation_x'] = random.randint(-args['max_translation_rate'], args['max_translation_rate'])
        #    args['translation_y'] = random.randint(-args['max_translation_rate'], args['max_translation_rate'])

            #args['translation_x'] = int(args['translation_x'] + trans_x*(chance+.25))
            #args['translation_x']  = clip(args['translation_x'], -args['max_translation_rate'], args['max_translation_rate'])

            #args['translation_y'] = int(args['translation_y'] + trans_y*(chance+.25))
            #args['translation_y']  = clip(args['translation_y'], -args['max_translation_rate'], args['max_translation_rate'])

      #else:
      #    args['last_beat_counter'] = args['last_beat_counter'] + 1

      #    if abs(args['rotation_rate']) > 0:
      #        args['rotation_rate'] = args['rotation_rate'] * args['decay']

      #    if abs(args['translation_x']) > 0:
      #      if args['translation_x'] > 0:
      #        args['translation_x'] = np.ceil(args['translation_x'] - args['last_beat_counter']*.025)
      #      else:
      #        args['translation_x'] = np.ceil(args['translation_x'] + args['last_beat_counter']*.025)

      #    if abs(args['translation_y']) > 0:
      #      if args['translation_y'] > 0:
      #        args['translation_y'] = np.ceil(args['translation_y'] - args['last_beat_counter']*.025)
      #      else:
      #        args['translation_y'] = np.ceil(args['translation_y'] + args['last_beat_counter']*.025)

    if ind > last_scene_change:
      args['rotation_rate'] = 0
      args['translation_x'] = 0
      args['translation_y'] = 0
      args['zoom_rate'] = .99

    for k, v in args.items():
      params_df.loc[ind, k] = v


  return params_df


In [7]:
#@title getting audio and creating project directory
import glob

base_path = '/content/drive/MyDrive/VisualLabs_experiment/'

### The base_filename, and youtube_link, need to be passed in as arguments
### These will change each time this code is run

base_filename = 'PriorityList'
#youtube_link = 'https://www.youtube.com/watch?v=Ug88EA7kFP0'

### This directory structure should be created within S3, all files saved in this
### Colab notebook should be uploaded to S3 in their respective directories

project_path = base_path + base_filename +'/'
frames_path = project_path + 'frames/'
audio_path = project_path + 'audio/'

for p in [project_path, frames_path, audio_path]:
  os.makedirs(p, exist_ok=True)

#filename = download_yt_audio(youtube_link, project_path, base_filename)
filename = glob.glob(project_path+base_filename+'*')[0]



In [ ]:
fps = 15

out_path = project_path

print('Performing source separation')
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"
# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!
separate(extensions, inp=out_path, outp=out_path)

# Move files to root audio directory, and get rid of extras
file_list = get_files(out_path + '/htdemucs/' + base_filename)
for f in file_list:
  shutil.copy(out_path + 'htdemucs/' + base_filename + '/' + f, audio_path + f.split('/')[-1] )
shutil.rmtree(out_path + 'htdemucs')

print('Compiling timeseries')
ts_df = compile_timeseries(base_filename, audio_path, fps)

# Extract beats from drums audio
print('Extracting beats and bpm')
spec_df = extract_spectrogram(audio_path + "/drums_converted_audio_file.wav", fps)
beats_df = get_beats(base_filename, spec_df)
bpm = get_bpm(filename)[0]

In [12]:
import whisper
import json

whisper_model = whisper.load_model("base")
lyrics_dic = whisper_model.transcribe(filename)

with open(project_path+'lyrics.json', 'w') as f:
  f.write(json.dumps(lyrics_dic))


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 49.4MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [37]:

params_df = get_video_params(ts_df, beats_df, bpm, fps, lyrics_dic, use_beats=False, use_lyrics=True)

# CSVs used to generate embeddings and parameters for image-to-image
ts_df.to_csv(project_path + 'audio_ts.csv', index=True)
params_df.to_csv(project_path + 'params_ts.csv', index=True)


In [38]:
params_df.head()


,beats,sentence_start,mov_av_audio,seed,guidance_scale,strength,steps,base_steps,base_zoom_rate,zoom_rate,...,max_rotation_rate,base_noise_amount,noise_amount,base_strength,beat_strength,base_translation_rate,max_translation_rate,decay,beat_scale,last_beat_counter
0,0,1.0,0.000000,237204576.0,7.5,1.0,30.0,30.0,0.99,0.99,...,4.0,0.06,0.0,0.6,0.8,7.0,10.0,0.99,1.01,0.0
1,0,0.0,0.000000,237204576.0,7.5,0.6,30.0,30.0,0.99,0.99,...,4.0,0.06,0.0,0.6,0.8,7.0,10.0,0.99,1.01,0.0
2,0,0.0,0.000000,544520625.0,7.5,0.6,30.0,30.0,0.99,0.99,...,4.0,0.06,0.0,0.6,0.8,7.0,10.0,0.99,1.01,0.0
3,0,0.0,0.000000,544520625.0,7.5,0.6,30.0,30.0,0.99,0.99,...,4.0,0.06,0.0,0.6,0.8,7.0,10.0,0.99,1.01,0.0
4,0,0.0,0.025473,575509579.0,7.5,0.6,30.0,30.0,0.99,0.99,...,4.0,0.06,0.0,0.6,0.8,7.0,10.0,0.99,1.01,0.0


In [2]:
import json

with open(project_path + 'inits.json') as f:
        init_dic = json.loads(f.read())

NameError: ignored

In [11]:
params_df.head()


,beats,mov_av_audio,seed,guidance_scale,strength,steps,base_zoom_rate,zoom_rate,rotation_rate,translation_x,...,noise_amount,base_strength,beat_strength,base_translation_rate,max_translation_rate,decay,beat_scale,last_beat_counter,cadence,generate_image
0,0,0.000000,194866411.0,7.5,0.4,30.0,1.01,1.01,0.0,0.0,...,0.0,0.4,0.8,7.0,10.0,0.99,1.01,0.0,2.0,True
1,0,0.000000,194866411.0,7.5,0.4,30.0,1.01,1.01,0.0,0.0,...,0.0,0.4,0.8,7.0,10.0,0.99,1.01,0.0,2.0,False
2,0,0.000000,194866411.0,7.5,0.4,30.0,1.01,1.01,0.0,0.0,...,0.0,0.4,0.8,7.0,10.0,0.99,1.01,0.0,2.0,True
3,0,0.000000,194866411.0,7.5,0.4,30.0,1.01,1.01,0.0,0.0,...,0.0,0.4,0.8,7.0,10.0,0.99,1.01,0.0,2.0,False
4,0,0.024259,268039828.0,7.5,0.4,30.0,1.01,1.01,0.0,0.0,...,0.0,0.4,0.8,7.0,10.0,0.99,1.01,0.0,2.0,True


In [ ]:
from google.colab import runtime
runtime.unassign()